In [1]:
import os
import re
import sys
import glob
import gensim
import numpy as np
import pandas as pd
from tqdm import tqdm
from uuid import uuid4
from functools import reduce
from multiprocessing import Pool
from sentence_transformers import SentenceTransformer
import faiss
import nltk
nltk.download('punkt_tab')
from nltk.corpus import stopwords
nltk.download('stopwords')

/tmp/ipykernel_12420/2762488061.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/hlopin/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-10-28 22:47:29.118153: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critic

True

In [2]:
from whoosh.index import create_in
from whoosh.fields import Schema, TEXT, ID
from whoosh.qparser import QueryParser
from sklearn.feature_extraction.text import TfidfVectorizer

## Извлечение текстов википедии из архива. Из архива достаём различные страницы из википедии, которые потом будем добавлять в базы данных

In [3]:
#!pip install wikiextractor

In [4]:
import wikiextractor

Скачиваем архив

In [5]:
#!wget http://dumps.wikimedia.org/ruwiki/latest/ruwiki-latest-pages-articles.xml.bz2

Разархивируем в ./wiki папку. Для начала лучше разархивировать только несколько папок с документами по википедии (AA,AB)

In [6]:
# !python3 -m wikiextractor.WikiExtractor -o ./data/wiki/ --no-templates --processes 8 ./ruwiki-latest-pages-articles.xml.bz2

In [7]:
# !python3 -m wikiextractor.extractPage -o ./data/wiki/ --no-templates --processes 8 ./ruwiki-latest-pages-articles.xml.bz2

## Функционал для парсинг текстов википедии

In [8]:
def _remove_non_printed_chars(string):
    reg = re.compile('[^a-zA-Zа-яА-ЯёЁ]')
    return reg.sub(' ', string)

def _remove_stop_words(string,sw=[]):
    return ' '.join([word if word not in sw else '' \
                     for word in string.strip().split(' ')])

def _trim_string(string):
    # remove extra spaces, remove trailing spaces, lower the case 
    return re.sub('\s+',' ',string).strip().lower()
    
def clean_string(string,
                 stop_words_list,
                 min_len=2,
                 max_len=30):

    string = _remove_non_printed_chars(string)
    string = _remove_stop_words(string,stop_words_list)
    string = _trim_string(string)
    # also remove short words, most likely containing addresses / crap / left-overs / etc remaining after removal
    # gensim mostly does the same as above, it is used here for simplicity
    string = ' '.join(gensim.utils.simple_preprocess(string,
                                                     min_len=min_len,
                                                     max_len=max_len))
    return string
    
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)
    
def remove_special_chars(text,char_list):
    for char in char_list:
        text=text.replace(char,'')
    return text.replace(u'\xa0', u' ')

def splitkeepsep(s, sep):
    cleaned = []
    s = re.split("(%s)" % re.escape(sep), s)
    for _ in s:
        if _!='' and _!=sep:
            cleaned.append(sep+_)
    return cleaned

def extract_url(text):
    pattern = 'http([^"]+)'
    match = re.search(pattern, text)
    if match:
        url = match.group(0)
        return url
    else:
        return ""

def create_vector(text):
    return model.encode(text, normalize_embeddings=True)
    
def process_wiki_files(wiki_file):
    chars = ['\n']
    global sw

    with open(wiki_file, encoding='utf-8') as f:
        content = f.read()

    articles = splitkeepsep(content,'<doc id=')
    df_texts = pd.DataFrame(columns=['article_uuid','url', 'title', 'article','proc_article','proc_len'])
    emds = []
    
    for article in articles:
        uuid_text = uuid4()

        url = extract_url(article)
        article = remove_html_tags(article)
        titleIndex = article.find('\n\n')
        title = article[1:article.find('\n\n')]

        article = remove_special_chars(article[titleIndex:], chars)        
        proc_article = clean_string(article,sw)
        proc_len = len(proc_article.split(' '))

        temp_df_texts = pd.DataFrame(
            {'article_uuid': [uuid_text],
             'url': url,
             'title': title,
             'article': article,
             'proc_article':proc_article,
             'proc_len':proc_len
            })
        df_texts = pd.concat([df_texts, temp_df_texts], ignore_index=True)

        emb = create_vector(proc_article)
        emds.append(emb)
    
    return df_texts, np.array(emds)

sw_en = set(stopwords.words('english'))
sw_ru = set(stopwords.words('russian'))
sw = list(sw_ru.union(sw_en))  

Получаем модель для эмбеддингов

In [9]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

/home/hlopin/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### Сохранение векторов текстов в векторную БД с обратным индексом (используем Faiss)

In [10]:

def saveEmbdsToVectorDB(embds, path):
    if not exists(path):
        index = faiss.IndexFlatL2(embds.shape[1]) 
        index = faiss.IndexIDMap(index)
        index.add_with_ids(embds, np.arange(0, embds.shape[0]))
        faiss.write_index(index, path)
    else:
        index = faiss.read_index(path)
        index.add_with_ids(embds, np.arange(index.ntotal, index.ntotal + embds.shape[0]))
        faiss.write_index(index, path)

### Получаем векторную БД

In [11]:
def getVectorDB(path):
    return faiss.read_index(path)

### Сохранение в текстовую БД с обратным индексом (для этого используется библиотека whoosh, которая реализует Lucene подобную индексацию)

In [12]:
# Создаем индексатор
from whoosh.index import create_in, open_dir
from os.path import exists
from whoosh.qparser import QueryParser

In [13]:
def addTextsToDB(pathDB, startInd, df_texts):
    schema = Schema(
        id=ID(stored=True),  # Поле для уникального идентификатора документа
        content=TEXT(stored=True),  # Поле для текста документа
        url=ID(stored=True)  # Поле для хранения URL
    )
    
    if not exists(pathDB):
        os.mkdir(pathDB)
        index = create_in(pathDB, schema)
    else:
        index = open_dir(pathDB)
    writer = index.writer()

    for doc_id in range(len(df_texts)):
        writer.add_document(id=str(startInd + doc_id), content=df_texts.iloc[doc_id]['proc_article'], url=df_texts.iloc[doc_id]['url'])
    writer.commit()

### Получаем текстовую БД

In [14]:
def getRevertedIndexTextDB(pathDB):
    return open_dir(pathDB)

### Поиск в текстовой БД по запросу

In [15]:
def textSearch_with_bm25_ranking(query, pathDB):
    index = getRevertedIndexTextDB(pathDB)
    with index.searcher() as searcher:
        query_parser = QueryParser("content", index.schema)
        parsed_query = query_parser.parse(query)
        print("Получился запрос вида: ", parsed_query)
        results = searcher.search(parsed_query)
        return np.array([(result['id'], result.score) for result in results])

Получаем тексты по индексам

In [16]:
# def get_rows_from_csv(filename, indices, dbLen):
#     rows_to_skip = set(range(dbLen + 1)) - set(indices)

#     # df = pd.DataFrame(columns=['article_uuid', 'url', 'title', 'article', 'proc_article', 'proc_len'])
    
#     df = pd.read_csv(
#         filename,
#         header=None,
#         skiprows=lambda x: x in rows_to_skip
#     )

#     # df = pd.concat([df, data], ignore_index=True)
    
#     # df = df.append(df_csv)
    
#     return df

In [17]:
def get_rows_from_csv(filename, indices):
    df = pd.read_csv(
        filename,
        header=None,
        skiprows=lambda x: x not in indices
    )
    
    return df

## Заполнение баз данных. (Объединяем всё вместе, парсим документы и сохраняем их в базы данных)

In [18]:
wikiFilesRootPath = "data/wiki"
vectorDBPath = 'data/data_bases/vectorDB.index'
textDBPath = "data/data_bases/revertedIndexTextDB"
textsCsvPath = "data/data_bases/texts.csv"

In [19]:
def process_file(file_path):
    print("Обрабатываю: ", file_path)
    df_texts, embds = process_wiki_files(file_path)
    df_texts.to_csv(textsCsvPath, mode='a', header=False)
    saveEmbdsToVectorDB(embds, vectorDBPath)
    index = getVectorDB(vectorDBPath)
    addTextsToDB(textDBPath, index.ntotal, df_texts)

Параллельно обрабатыватексты тексты и сохраяем результаты в базы данных

Вместо наполнения баз данных самому можно скачать примеры отсюда: https://disk.yandex.ru/d/tPGZGoJVTPyNvQ

In [20]:
# from concurrent.futures import ThreadPoolExecutor

# files_to_process = []
# for dirpath, dirnames, filenames in os.walk(wikiFilesRootPath):
#     for filename in filenames:
#         file_path = os.path.join(dirpath, filename)
#         files_to_process.append(file_path)

#  # Используем ThreadPoolExecutor для параллельной обработки файлов
# with ThreadPoolExecutor(max_workers=8) as executor:  # Количество рабочих потоков можно настроить
#     futures = {executor.submit(process_file, file_path): file_path for file_path in files_to_process}
    
#     # Ждем завершения всех задач
#     for future in concurrent.futures.as_completed(futures):
#         file_path = futures[future]
#         try:
#             data = future.result()
#         except Exception as exc:
#             print(f'Ошибка при обработке файла {file_path}: {exc}')

Проверка на адекватность векторной БД

In [23]:
index = getVectorDB(vectorDBPath)

texts = get_rows_from_csv(textsCsvPath, range(30))[5][:30]
first_30_vectors = model.encode(texts, normalize_embeddings=True)
    
# Находим соседей для первых 30 векторов в БД
D, I = index.search(np.array(first_30_vectors), 12)
# Должны быть те же индексы от 0 до 29. То есть те же векторы, для которых искали соседей. (там где не совпадает, это потомучто вместо текста Nan, снизу видно)
print("Индексы похожих документов:", I[0:30, 0])


Индексы похожих документов: [    0     1     2     3     4     5     6     7     8     9    10    11
    12    13    14 14932    16    17    18    19    20    21    22    23
    24    25    26 14932    28    29]


In [24]:
texts

0     го тфрид ви льгельм ле йбниц немецкий философ ...
1     тит лукре ций кар часто просто лукреций родилс...
2     ма ртин лю тер монашеское имя августин ноября ...
3     марк авре лий антони апреля рим марта виндобон...
4     фри дрих ви льгельм ни цше октября рёккен вайс...
5     пармени эле ок ок древнегреческий философ глав...
6     пи ндар фивы аргос самых значительных лирическ...
7     блез паска ль французский литератор теолог кла...
8     пифаго са мосский около годов древнегреческий ...
9     плато афинский философ классического периода д...
10    протаго ок ок абдер древнегреческий философ ви...
11    лу ций анне се ка се ка мла дший просто се ка ...
12    сокра около года афины год афины древнегреческ...
13    влади мир серге евич соловьёв русский религиоз...
14    бенеди кт спино рождении бару спино ноября амс...
15                                                  NaN
16    ма ртин ха йдеггер сентября месскирх великое г...
17    себастье рок никола де шамфо апреля клермо

## Процесс поиска

Здесь мы находим kDocuments релевантных документов с помощью векторного поиска и далее ранжируем

In [ ]:
!pip install pymorphy2 rank_bm25

In [20]:
import nltk
from nltk.stem.snowball import SnowballStemmer

# Загрузка необходимых ресурсов
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [41]:
from abc import ABC, abstractmethod
from rank_bm25 import BM25Okapi
from sentence_transformers import CrossEncoder


class DocsRanker(ABC):
    @abstractmethod
    def rankDocuments(self, query, docs):
        pass


class Bm25Ranker(DocsRanker):
    # preprocess_func: переобразует запрос и документ в список слов
    def __init__(self, preprocess_func = None) -> None:
        self.preprocess_func = preprocess_func

    def rankDocuments(self, query, docs):
        if self.preprocess_func is None:
            self.preprocess_func = lambda doc: doc.split()
        tokenized_corpus = [self.preprocess_func(doc) for doc in docs]
        bm25 = BM25Okapi(tokenized_corpus)
        tokenized_query = self.preprocess_func(query)
        return bm25.get_scores(tokenized_query)

def lemmatize(doc):
    morph = pymorphy2.MorphAnalyzer()
    return [morph.parse(word)[0].normal_form for word in doc.split()]

def stem(doc):
   stemmer = SnowballStemmer("russian")
   words = nltk.word_tokenize(doc, language="russian")
   return [stemmer.stem(word) for word in words]


class CrossEncoderRanker(DocsRanker):
    def __init__(self) -> None:
        self.reranker_model = CrossEncoder('DiTy/cross-encoder-russian-msmarco', max_length=512, device='cuda')

    def rankDocuments(self, query, docs):
        return np.array([self.reranker_model.predict([[query, doc]])[0] for doc in docs])

In [42]:
from rank_bm25 import BM25Okapi
import pymorphy2

def findVectorsIndexes(query, encoder, kDocuments):
  index = getVectorDB(vectorDBPath)
  queryEmbd = encoder.encode(query, normalize_embeddings=True)
  D, I = index.search(np.array([queryEmbd]), kDocuments)
  return I[0]

def retrieveDocsAndUrls(indexes):
  urlsAndDocs = get_rows_from_csv(textsCsvPath, indexes)[[2, 5]]
  urlsAndDocs = urlsAndDocs.fillna('stub')
  return urlsAndDocs[2], urlsAndDocs[5]

def rankDocuments(query, indexes, ranker):
    urls, docs = retrieveDocsAndUrls(indexes)
    doc_scores = ranker.rankDocuments(query, docs)
    sorted_idx = np.argsort(doc_scores)
    return list(docs.iloc[sorted_idx[::-1]]), list(urls.iloc[sorted_idx[::-1]]), doc_scores[sorted_idx[::-1]]

def getSortedDocumentsWithUrls(query, encoder, kDocuments, ranker):
  indexes = findVectorsIndexes(query, encoder, kDocuments)
  return rankDocuments(query, indexes, ranker)

In [43]:
query = "Самое крупное государство"
kDocuments = 50
docs, urls, bm25_scores = getSortedDocumentsWithUrls(query, model, kDocuments, Bm25Ranker())
df = pd.DataFrame({'relevant docs': docs, 'bm25 scores': bm25_scores})
print(df)

                                        relevant docs  bm25 scores
0   сан мари официально респу блика сан мари также...     3.762134
1   соединённые штаты америки государство располож...     3.184887
2   герма ния полное официальное название федерати...     2.557143
3   брази лия официально федерати вная респу блика...     2.413398
4   евра зия крупнейший шести материков земле площ...     2.406947
5   соединённые шта аме рики сокращённо сша соедин...     2.208574
6   госуда рство политическая форма устройства общ...     1.865657
7   нды ндские южноамерика нские кордилье ры самая...     1.586740
8   чад официальное название респу блика чад госуд...     1.502173
9   индоне зия официальное название респу блика ин...     1.494897
10  респу блика коре nmin uk коре йская респу блик...     1.487920
11  лихтенште йн официально кня жество лихтенште й...     1.484579
12  бе льгия полная официальная форма короле вство...     1.479673
13  ниге рия полное название федерати вная респу б...     1.46

In [44]:
query = "Самое крупное государство"
kDocuments = 50
docs, urls, bm25_scores = getSortedDocumentsWithUrls(query, model, kDocuments, Bm25Ranker(lemmatize))
df = pd.DataFrame({'relevant docs': docs, 'bm25 scores': bm25_scores})
print(df)

                                        relevant docs  bm25 scores
0   евра зия крупнейший шести материков земле площ...     4.947120
1   чад официальное название респу блика чад госуд...     4.867134
2   бе льгия полная официальная форма короле вство...     4.812596
3   герма ния полное официальное название федерати...     4.803103
4   соединённые шта аме рики сокращённо сша соедин...     4.792313
5   зия крупнейшая часть света территории численно...     4.775716
6   брази лия официально федерати вная респу блика...     4.709885
7   росси росси йская федера ция сокр рф государст...     4.694046
8   исла ндия страна льдов ледяная страна островно...     4.680309
9   ботсва полная официальная форма респу блика бо...     4.644851
10  кана государство северной америке крупнейшее п...     4.628618
11  индоне зия официальное название респу блика ин...     4.627982
12  калифо рния штат сша расположенный западном по...     4.604532
13  золота орда улус джучи многонациональное госуд...     4.43

In [45]:
query = "Самое крупное государство"
kDocuments = 50
docs, urls, bm25_scores = getSortedDocumentsWithUrls(query, model, kDocuments, Bm25Ranker(stem))
df = pd.DataFrame({'relevant docs': docs, 'bm25 scores': bm25_scores})
print(df)

                                        relevant docs  bm25 scores
0   чад официальное название респу блика чад госуд...     4.972396
1   евра зия крупнейший шести материков земле площ...     4.928234
2   герма ния полное официальное название федерати...     4.887189
3   росси росси йская федера ция сокр рф государст...     4.821010
4   бе льгия полная официальная форма короле вство...     4.808878
5   соединённые шта аме рики сокращённо сша соедин...     4.792408
6   зия крупнейшая часть света территории численно...     4.757485
7   индоне зия официальное название респу блика ин...     4.737161
8   брази лия официально федерати вная респу блика...     4.735336
9   золота орда улус джучи многонациональное госуд...     4.685704
10  исла ндия страна льдов ледяная страна островно...     4.685329
11  ботсва полная официальная форма респу блика бо...     4.675351
12  кана государство северной америке крупнейшее п...     4.665088
13  лихтенште йн официально кня жество лихтенште й...     4.60

In [46]:
query = "Самое крупное государство"
kDocuments = 50
docs, urls, scores = getSortedDocumentsWithUrls(query, model, kDocuments, CrossEncoderRanker())
df = pd.DataFrame({'relevant docs': docs, 'scores': scores})
print(df)

                                        relevant docs    scores
0   украи государство восточной центральной европе...  0.046154
1   кана государство северной америке крупнейшее п...  0.032627
2   золота орда улус джучи многонациональное госуд...  0.030256
3   солт лейк си ти столица самый населённый самый...  0.030136
4   зия крупнейшая часть света территории численно...  0.024219
5   росси росси йская федера ция сокр рф государст...  0.018571
6   та штат сша группе горных штатов расположен ра...  0.012329
7   ло ндон столица крупнейший город англии велико...  0.005403
8   калифо рния штат сша расположенный западном по...  0.004397
9   сан мари официально респу блика сан мари также...  0.003206
10  госуда рство политическая форма устройства общ...  0.002975
11  евра зия крупнейший шести материков земле площ...  0.002919
12  нами бия официальное название респу блика нами...  0.002612
13  нды ндские южноамерика нские кордилье ры самая...  0.001995
14  нью йо рк сокр nyc крупнейший город 

## Примеры использования различных БД для поиска

In [32]:
df_texts, embds = process_wiki_files("./data/wiki/AA/wiki_00")

In [33]:
df_texts

,article_uuid,url,title,article,proc_article,proc_len
0,0db8c75d-bc63-4894-84cf-b81b61244528,https://ru.wikipedia.org/wiki?curid=4,Базовая статья,,,1
1,229fd47a-1930-4abd-921f-d5488fa5ae6c,https://ru.wikipedia.org/wiki?curid=7,Литва,"Литва́ ( ), официальное название — Лито́вская ...",литва официальное название лито вская респу бл...,4324
2,8d5864ac-ca16-482d-bb65-2853a73c2f42,https://ru.wikipedia.org/wiki?curid=9,Россия,"Росси́я, или Росси́йская Федера́ция (сокр. РФ)...",росси росси йская федера ция сокр рф государст...,14331
3,d3007aab-0973-4283-9c43-00ca24b463c4,https://ru.wikipedia.org/wiki?curid=10,Слоновые,"Слоно́вые, или слоны́ , — семейство класса мле...",слоно вые слоны семейство класса млекопитающих...,1580
4,3e760eeb-23e5-4aab-a7bb-0f621f887160,https://ru.wikipedia.org/wiki?curid=11,Мамонты,Ма́монты () — вымерший род млекопитающих отряд...,ма монты вымерший род млекопитающих отряда хоб...,4193
5,8fbe6299-2675-41a5-a067-c9051c0f5ba6,https://ru.wikipedia.org/wiki?curid=15,Красная книга,Кра́сная кни́га — аннотированный список редких...,кра сная кни га аннотированный список редких н...,806
6,1c1db8e4-8762-4dd8-9f3b-576a1f07c450,https://ru.wikipedia.org/wiki?curid=16,Соционика,Социо́ника — псевдонаучная концепция типов лич...,социо ника псевдонаучная концепция типов лично...,2121
7,3ca77c2e-ff1d-45b7-b5df-981b83a059f6,https://ru.wikipedia.org/wiki?curid=18,Школа,"Шко́ла (через , из , от — «досуг») — учебное ...",шко ла досуг учебное заведение получения общег...,1782
8,af70ba8b-82b7-4f01-9e5a-c95b9034bddd,https://ru.wikipedia.org/wiki?curid=20,Лингвистика,"Лингви́стика (от «язык»), языкозна́ние, языко...",лингви стика язык языкозна ние языкове дение н...,529
9,4cf782b0-efdf-4573-83ab-4028ebecff25,https://ru.wikipedia.org/wiki?curid=21,Социология,Социоло́гия (от и ) ― наука о совместной жизни...,социоло гия наука совместной жизни групп сообщ...,10390


### Сохранение в базы данных извлечённых текстов и их эмбеддингов

#### Сохранение/дополнение векторной БД Faiss

In [34]:
# saveEmbdsToVectorDB(embds, './data/data_bases/vectorDBTest.index')

#### Чтение из векторной БД Faiss

In [35]:
index = getVectorDB('./data/data_bases/vectorDB.index')

In [36]:
index.ntotal

15035

### Поиск документов по запросу

In [37]:
D, I = index.search(np.array([model.encode("россия", normalize_embeddings=True)]), 10)
print("Индексы похожих документов:", I[0])
# print(D)
print()
print("А вот сами документы:")
test_df = get_rows_from_csv(textsCsvPath, I[0])
print(test_df[5])

Индексы похожих документов: [14715 12176 12177   426  6799  7109 11139 11127 11421 13518]

А вот сами документы:
0                              уфа топоним россии
1    чулы название географических объектов россии
2                                       гу рьевск
3       ме льница населённые пункты россияукраина
4      тро ицк название населённых пунктов россии
5                        потёмкин русская фамилия
6                        важнейшие события россии
7                        важнейшие события россии
8                        ро занов русская фамилия
9                    междуре ченск топоним россии
Name: 5, dtype: object


### Пример сохранения в текстовую БД

In [38]:
# addTextsToDB("data/data_bases/revertedIndexTextDBTest", 0, df_texts)

### Чтение из текстовой БД

In [39]:
textIndex = getRevertedIndexTextDB("data/data_bases/revertedIndexTextDB")

### Поиски по запросу

In [40]:
query = "россия"
k = 10

resulsIdAndBm25 = textSearch_with_bm25_ranking(query, "data/data_bases/revertedIndexTextDB")
print()
print("ID документов по результатам текстового поиску обратным индексом:", resulsIdAndBm25[:k,0])
print("Значения BM25:", resulsIdAndBm25[:,1])

print()
print("А вот названия самих документов:")
test_df = get_rows_from_csv(textsCsvPath, [int(id) for id in resulsIdAndBm25[:k,0]], index.ntotal)
print(test_df[5])

Получился запрос вида:  content:россия

ID документов по результатам текстового поиску обратным индексом: ['10849' '10850' '10861' '9468' '9434' '12079' '10867' '8192' '10855'
 '908']
Значения BM25: ['8.09784968001708' '7.631559273474731' '7.5425688565232845'
 '6.993329497909029' '6.970867853166101' '6.680234717070014'
 '6.605632593634342' '6.5571022102496395' '6.442766098981979'
 '6.371415928511168']

А вот названия самих документов:
0    праздники памятные дни см также события см так...
1    кисло химические соединения способные отдавать...
2    львовский национальный университет имени ивана...
3    десяти чная дробь разновидность дроби которая ...
4    альберт сент дьёрдьи сентября будапешт октября...
5    фрирайд свободная езда спуск правило естествен...
6    эндрю юр мая глазго шотландия января лондон шо...
7    галаха алаха халаха ашкеназ ало хо гало хо тра...
8    прогре сс движение вперёд успех направление ра...
9    ба бушкин года мы совск город районного значен...
Name: 5, dt

In [41]:
query = "русь"
k = 10

resulsIdAndBm25 = textSearch_with_bm25_ranking(query, "data/data_bases/revertedIndexTextDB")
print()
print("ID документы по результатам текстового поиску обратным индексом:", resulsIdAndBm25[:k,0])
print("Значения BM25:", resulsIdAndBm25[:,1])

print()
print("А вот названия самих документов:")
test_df = get_rows_from_csv(textsCsvPath, [int(id) for id in resulsIdAndBm25[:k,0]], index.ntotal)
print(test_df[5])

Получился запрос вида:  content:русь

ID документы по результатам текстового поиску обратным индексом: ['649' '7188' '879' '8275' '4228' '4265' '4266' '4267' '4268' '4283']
Значения BM25: ['8.975858721012742' '8.931509280655012' '7.6977715785940095'
 '7.6977715785940095' '7.6733597979402015' '7.6733597979402015'
 '7.6733597979402015' '7.6733597979402015' '7.6733597979402015'
 '7.6733597979402015']

А вот названия самих документов:
0                                                  NaN
1                                                  NaN
2                родились см также скончались см также
3                родились см также скончались см также
4                родились см также скончались см также
5                родились см также скончались см также
6                родились см также скончались см также
7                родились см также скончались см также
8    имму нная систе ма система биологических струк...
9    ваа гн вахагн древнеармянской мифологии бог ог...
Name: 5, dtype:

# Итог

### Что имеем.

У нас есть архив(ы) со страницами с википедии, из которых мы составляем:
1) Векторную БД (Faiss)
2) Тестовую БД, работающую на обратном индексе (библиотека Whoosh, которая работает по принципу Lucene)
3) csv файл, в котором хранятся все спаршенные страницы

Пока что всё таким образом (храним лишнюю csv, которая к тому же содержит полные тексты документов), потому что только начинаем разрабать и тестировать поисковик. Поэтому нужно от этого будет избавиться и сделать всё эффективнее.



**Как сейчас ведётся поиск**.
1) Получаем текстовый запрос
2) Переводим его в вектор
3) Находим k документов из векторной БД, которые наибольшим образом похожи на запрос

### Что нужно сделать дальше

Нужно как-то отранжировать полученные k документов из векторной БД. 

Моё изначальное предложение было - отранжировать текстово с использованием алгоритма BM25. Можно так и сделать. 

Или же предлагайте новые варианты.